<img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px">

<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>Projet Data Science</h1></center>
<center><h2>Création de jeux de données</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">

<blockquote>
Dans cette partie, nous allons voir comment créer des jeux de données intermédiaires et les stocker pour des analyses plus tard. Persister certains jeux de données permet à la fois d'aller plus vite dans l'analyse des données mais aussi de pouvoir diviser le travail au sein d'une équipe.
</blockquote>

* importer <code>pandas</code> et charger le jeu de données <code>events.csv</code>

In [1]:
# Insérer le code ici


In [2]:
# Insérer le code ici

import pandas as pd 

df = pd.read_csv('events.csv')
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


<hr style="border-width:2px;border-color:#75DFC1">
<center><h2>Constitution du jeu de données visiteurs</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">

<blockquote>
Dans cette partie, nous allons créer un jeu de données qui comprends les données aggrégées à la maille utilisateurs. Ceci devrait nous permettre une première approche du jeu de données et de comprendre quels sont les comportements moyens.<br>
Pour ceci, nous allons devoir retravailler le jeu de données afin d'y faire apparaître différentes choses. <br><br>
Dans un premier temps, nous voulons garder le nombre d'événements différents par utilisateur. Une manière assez simple consiste à binariser la colonne <code>event</code> et ensuite faire la somme de ces colonnes lors de l'étape d'aggrégation.

</blockquote>

* dupliquer la colonne <code>event</code> en <code>event2</code>
* binariser la colonne <code>event2</code> à l'aide de la fonction <code>pd.get_dummies</code>

In [3]:
# Insérer le code ici


In [4]:
# Insérer le code ici

df['event2'] = df['event']
df = pd.get_dummies(df, columns=['event2'], prefix=['event'] )
df.head()

,timestamp,visitorid,event,itemid,transactionid,event_addtocart,event_transaction,event_view
0,1433221332117,257597,view,355908,NaN,0,0,1
1,1433224214164,992329,view,248676,NaN,0,0,1
2,1433221999827,111016,view,318965,NaN,0,0,1
3,1433221955914,483717,view,253185,NaN,0,0,1
4,1433221337106,951259,view,367447,NaN,0,0,1


<blockquote>
    Il nous faut aussi transformer la colonne <code>timestamp</code> en colonne de type <code>datetime</code>: cela nous permettra plus de possibilités:
    
</blockquote>

* créer une colonne <code>datetime</code> qui contient la date en utilisant <code>datetime.fromtimestamp</code> du package <code>datetime</code> et <code>apply</code>

In [5]:
# Insérer le code ici


In [6]:
# Insérer le code ici

import datetime

df['datetime'] = df['timestamp'].apply(lambda ts: datetime.datetime.fromtimestamp(ts / 1000))

print(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 9 columns):
timestamp            int64
visitorid            int64
event                object
itemid               int64
transactionid        float64
event_addtocart      uint8
event_transaction    uint8
event_view           uint8
datetime             datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(1), uint8(3)
memory usage: 134.0+ MB
None


,timestamp,visitorid,event,itemid,transactionid,event_addtocart,event_transaction,event_view,datetime
0,1433221332117,257597,view,355908,NaN,0,0,1,2015-06-02 07:02:12.117
1,1433224214164,992329,view,248676,NaN,0,0,1,2015-06-02 07:50:14.164
2,1433221999827,111016,view,318965,NaN,0,0,1,2015-06-02 07:13:19.827
3,1433221955914,483717,view,253185,NaN,0,0,1,2015-06-02 07:12:35.914
4,1433221337106,951259,view,367447,NaN,0,0,1,2015-06-02 07:02:17.106


<blockquote>
    Nous allons ensuite aggréger par <code>visitorid</code> mais nous devons d'abord réfléchir aux fonctions d'aggrégations: 
    <ul>
        <li><code>event_view</code>: somme pour obtenir le nombre de produits vus</li>
        <li><code>event_addtocart</code>: somme pour obtenir le nombre de produits ajoutés au panier</li>
        <li><code>event_transaction</code>: somme pour obtenir le nombre de produits achetés</li>
        <li><code>datetime</code>: minimum et maximum pour obtenir la durée d'activité sur le site internet (on pourrait garder uniquement la différence en jours, heures ou semaines mais il peut être intéressant d'avoir aussi le début et la fin.</li>
        <li><code>transaction_id</code>: comptage du nombre de transaction différentes</li>.
    </ul>
Il serait intéressant de récupérer aussi le nombre de produits différents par type d'événement pour un utilisateur. Comme les produits sont différenciés par leur id, on peut créer trois colonnes supplémentaires, une pour chaque type d'événement. Elle ne contiendra rien sauf si la ligne correspond au type d'observation de la colonne, auquel cas elle contiendra le <code>itemid</code>. Il suffira ensuite de compter le nombre de produits différents dans chacune des colonnes
</blockquote>

* créer 3 colonnes <code>itemid_view</code>, <code>itemid_addtocart</code> et <code>itemid_transaction</code> qui ne contient que <code>0</code>
* remplir chacune de ces colonnes avec le contenu de <code>productid</code> en fonction de si l'événement correspond.

In [7]:
# Insérer le code ici


In [8]:
# Insérer le code ici

new_columns = ['itemid_view', 'itemid_addtocart', 'itemid_transaction']
events = ['view', 'addtocart', 'transaction']

for col, event in zip(new_columns, events): 
    df[col] = 0
    df.loc[df['event'] == event, col] = df['itemid'][df['event'] == event]

df.head()

,timestamp,visitorid,event,itemid,transactionid,event_addtocart,event_transaction,event_view,datetime,itemid_view,itemid_addtocart,itemid_transaction
0,1433221332117,257597,view,355908,NaN,0,0,1,2015-06-02 07:02:12.117,355908,0,0
1,1433224214164,992329,view,248676,NaN,0,0,1,2015-06-02 07:50:14.164,248676,0,0
2,1433221999827,111016,view,318965,NaN,0,0,1,2015-06-02 07:13:19.827,318965,0,0
3,1433221955914,483717,view,253185,NaN,0,0,1,2015-06-02 07:12:35.914,253185,0,0
4,1433221337106,951259,view,367447,NaN,0,0,1,2015-06-02 07:02:17.106,367447,0,0


* créer une fonction <code>get_number_of_unique_elements</code> qui prend en entrée un élément de type <code>pd.Series</code> et qui retourne le nombre d'éléments différents. (on pourra la fonction <code>unique</code> de <code>numpy</code>) sans prendre en compte les valeurs nulles.

In [9]:
# Insérer le code ici


In [10]:
# Insérer le code ici

import numpy as np

def get_number_of_unique_elements(series):
    series_unique = series.unique()
    
    return len(series_unique)
    

<blockquote>
Pour appliquer différentes fonctions d'aggrégations aux différentes colonnes, on utilise un dictionnaire: les clefs sont les colonnes de notre <code>DataFrame</code> et les valeurs les fonctions utilisées: 

```python 
{
    'col1': sum,
    'col2': my_function,
    'col3': lambda x: some_function(x),
    'col4': [my_first_function, my_second_function]
}
```

</blockquote>

* construire le dictionnaire d'aggrégation pour avoir les informations données plus tôt


In [11]:
# Insérer le code ici


In [12]:
# Insérer le code ici

aggregation = {
    'transactionid': get_number_of_unique_elements,
    'event_view': sum,
    'event_addtocart': sum,
    'event_transaction': sum,
    'datetime': [min, max],
    'itemid_view': get_number_of_unique_elements,
    'itemid_addtocart': get_number_of_unique_elements,
    'itemid_transaction': get_number_of_unique_elements,
    'itemid': get_number_of_unique_elements
}

* effectuer l'aggrégation par <code>visitorid</code> grâce à la méthode <code>groupby</code> et mettre le résultat dans un nouveau <code>DataFrame</code> nommé <code>df_visitor</code>.

In [13]:
# Insérer le code ici


In [14]:
# Insérer le code ici

df_visitor = df.groupby(by=['visitorid']).agg(aggregation)
df_visitor.head()

transactionid event_view event_addtocart  \
          get_number_of_unique_elements        sum             sum   
visitorid                                                            
0                                   1.0        3.0             0.0   
1                                   1.0        1.0             0.0   
2                                   1.0        8.0             0.0   
3                                   1.0        1.0             0.0   
4                                   1.0        1.0             0.0   

          event_transaction                datetime                          \
                        sum                     min                     max   
visitorid                                                                     
0                       0.0 2015-09-11 22:49:49.439 2015-09-11 22:55:17.175   
1                       0.0 2015-08-13 19:46:06.444 2015-08-13 19:46:06.444   
2                       0.0 2015-08-07 19:51:44.567 2015-08-07 20:20:57.845   
3                       0.0 2015-08-01 09:10:35.296 2015-08-01 09:10:35.296   
4                       0.0 2015-09-15 23:24:27.167 2015-09-15 23:24:27.167   

                            itemid_view              itemid_addtocart  \
          get_number_of_unique_elements get_number_of_unique_elements   
visitorid                                                               
0                                     3                             1   
1                                     1                             1   
2                                     4                             1   
3                                     1                             1   
4                                     1                             1   

                     itemid_transaction                        itemid  
          get_number_of_unique_elements get_number_of_unique_elements  
visitorid                                                              
0                                     1                             3  
1                                     1                             1  
2                                     1                             4  
3                                     1                             1  
4                                     1                             1

<blockquote>
    Ce jeu de données a pris du temps à se faire car l'étape d'aggrégation est particuliérement chronophage. Mais maintenant que ce jeu de données est créé, nous pouvons le sauvegarder sur le disque afin de limiter les opérations la prochaine fois que nous voulons étudier les visiteurs. 
    Nous allons juste retravailler le jeu de données afin de rendre plus clairs les noms des colonnes. 
</blockquote>

* éxécuter la cellule suivante pour renommer les colonnes: 

In [15]:
# Exécuter cette cellule

new_column_names = [
    'nb_unique_transactions',
    'nb_view_events',
    'nb_addtocart_events',
    'nb_transaction_events',
    'min_datetime',
    'max_datetime',
    'nb_view_unique_products',
    'nb_addtocart_unique_products',
    'nb_transaction_unique_products',
    'nb_unique_products'
]

df_visitor.columns = new_column_names  # on renomme les colonnes
df_visitor.head()

,nb_unique_transactions,nb_view_events,nb_addtocart_events,nb_transaction_events,min_datetime,max_datetime,nb_view_unique_products,nb_addtocart_unique_products,nb_transaction_unique_products,nb_unique_products
visitorid,,,,,,,,,,
0,1.0,3.0,0.0,0.0,2015-09-11 22:49:49.439,2015-09-11 22:55:17.175,3,1,1,3
1,1.0,1.0,0.0,0.0,2015-08-13 19:46:06.444,2015-08-13 19:46:06.444,1,1,1,1
2,1.0,8.0,0.0,0.0,2015-08-07 19:51:44.567,2015-08-07 20:20:57.845,4,1,1,4
3,1.0,1.0,0.0,0.0,2015-08-01 09:10:35.296,2015-08-01 09:10:35.296,1,1,1,1
4,1.0,1.0,0.0,0.0,2015-09-15 23:24:27.167,2015-09-15 23:24:27.167,1,1,1,1


<blockquote>Notons que la colonne transactions uniques contient une valeur pour tous les <code>na</code>. Il faut retraiter cette variable avant de sauvegarder. Il suffit d'enlever 1 à toute cette colonne.</blockquote>

* remplacer la colonne par la bonne valeur

In [16]:
# Insérer le code ici


In [17]:
# Insérer le code ici

df_visitor['nb_unique_transactions'] = df_visitor['nb_unique_transactions'] -1

<blockquote>
Il faut maintenant sauvegarder les données pour plus tard.
</blockquote>

* éxecuter la cellule suivante

In [18]:
# Exécuter cette cellule

df_visitor.to_csv('visitor_data.csv')

<hr style="border-width:2px;border-color:#75DFC1">
<center><h2>Constitution du jeu de données produits</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">

<blockquote>
Nous allons à présent créer un jeu de données similaires pour les produits. Il faut donc créer (ou réutiliser) des variables pour aggréger les données à la maille produit. <br>
Nous voulons savoir pour chaque produit: 
<ul>
    <li>le nombre de fois où il a été vu, ajouté au panier, acheté.</li>
    <li>combien de visiteurs uniques sont impliqués dans une chacune de ces actions</li>
    <li>les dates minimum et maximum auxquelles, ils ont subi des actions (ce qui pourrait impliquer leur présence ou absence sur le site notamment).</li>
</ul>
</blockquote>

* réaliser les opération qui permettent d'obtenir ces données dans un dataset <code>df_item</code>
* le nom des colonnes à garder est donné ci-dessous
* sauvegarder le jeu de données obtenu dans un fichier appelé <code>item_data.csv</code>


In [19]:
# Insérer votre code ici

# nom des colonnes 
new_column_names = [
    'nb_unique_transactions',
    'nb_view_events',
    'nb_addtocart_events',
    'nb_transaction_events',
    'min_datetime',
    'max_datetime',
    'nb_view_unique_visitors',
    'nb_addtocart_unique_visitors',
    'nb_transaction_unique_visitors',
    'nb_unique_visitors'
]


In [20]:
# Insérer le code ici

# nom des colonnes 
new_column_names = [
    'nb_unique_transactions',
    'nb_view_events',
    'nb_addtocart_events',
    'nb_transaction_events',
    'min_datetime',
    'max_datetime',
    'nb_view_unique_products',
    'nb_addtocart_unique_products',
    'nb_transaction_unique_products',
    'nb_unique_products'
]

# calcul du nombre de visiteurs uniques en fonction par action

new_columns = ['visitorid_view', 'visitorid_addtocart', 'visitorid_transaction']
events = ['view', 'addtocart', 'transaction']

for col, event in zip(new_columns, events): 
    df[col] = 0
    df.loc[df['event'] == event, col] = df['visitorid'][df['event'] == event]
    
# construction du dictionnaire d'aggrégation

aggregation = {
    'transactionid': get_number_of_unique_elements,
    'event_view': sum,
    'event_addtocart': sum,
    'event_transaction': sum,
    'datetime': [min, max],
    'visitorid_view': get_number_of_unique_elements,
    'visitorid_addtocart': get_number_of_unique_elements,
    'visitorid_transaction': get_number_of_unique_elements,
    'visitorid': get_number_of_unique_elements
}

# aggregation
df_item = df.groupby(['itemid']).agg(aggregation)
df_item.columns = new_column_names
print(df_item.head())

df_item.to_csv('item_data.csv')

        nb_unique_transactions  nb_view_events  nb_addtocart_events  \
itemid                                                                
3                          1.0             2.0                  0.0   
4                          1.0             3.0                  0.0   
6                          1.0            29.0                  0.0   
9                          1.0             2.0                  0.0   
15                         2.0            18.0                  3.0   

        nb_transaction_events            min_datetime            max_datetime  \
itemid                                                                          
3                           0 2015-08-18 20:30:40.493 2015-08-31 16:39:02.792   
4                           0 2015-06-30 09:03:11.545 2015-09-16 01:22:44.099   
6                           0 2015-05-06 22:33:13.122 2015-08-27 19:09:02.814   
9                           0 2015-05-14 00:53:14.505 2015-07-09 03:35:08.315   
15              

<hr style="border-width:2px;border-color:#75DFC1">
<center><h2>Constitution du jeu de données transactions</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">
<blockquote>
Dans cette partie, nous allons finalement analyser les différentes transactions:
<ul>
    <li>compter le nombre de produits uniques par transaction</li>
    <li>compter le nombre de produits par transaction</li>
    <li>l'heure à laquelle s'est faite la transaction</li>
</ul>
</blockquote>
    
* créer un DataFrame <code>df_transaction</code> qui contient ces informations avec le <code>transactionid</code> en index. 
* les colonnes devraient être <code>nb_unique_items</code>, <code>nb_items</code> et <code>datetime</code>
* sauvegarder les données dans un fichier nommé <code>transaction_data.csv</code>


In [21]:
# Insérer le code ici


In [22]:
# Insérer le code ici

# on supprime déjà les lignes sans transactionid

df_transaction = df[df['transactionid'].notna()]


df_transaction = df_transaction.groupby(['transactionid']).agg(
    {
        'itemid': [get_number_of_unique_elements, len],
        'datetime': min
    }
)

df_transaction.columns = ['nb_unique_items', 'nb_items', 'datetime']

df_transaction.to_csv('transaction_data.csv')
df_transaction.head()

,nb_unique_items,nb_items,datetime
transactionid,,,
0.0,1,1,2015-06-15 23:36:37.081
1.0,1,1,2015-07-14 20:30:08.851
2.0,1,1,2015-06-03 20:02:29.857
3.0,1,1,2015-06-22 02:12:41.626
4.0,1,1,2015-06-08 00:40:24.966


<blockquote>
Nous avons donc construit 3 jeux de données qui nous permettront de comprendre les distributions de nos données, les comportements moyens, ... C'est ce que nous ferons dans la suite!
</blockquote>